# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 23 2022 11:46AM,240296,10,PRF-30861-BO,Bio-PRF,Released
1,Jun 23 2022 11:40AM,240295,21,618597,"NBTY Global, Inc.",Released
2,Jun 23 2022 11:38AM,240294,10,SONSB0001829,"Nextsource Biotechnology, LLC",Released
3,Jun 23 2022 11:38AM,240294,10,SOTAH0000711,"Nextsource Biotechnology, LLC",Released
4,Jun 23 2022 11:38AM,240294,10,SONSB0001830,"Nextsource Biotechnology, LLC",Released
5,Jun 23 2022 11:38AM,240294,10,SONSB0001831,"Nextsource Biotechnology, LLC",Released
6,Jun 23 2022 11:38AM,240294,10,SOTAH0000712,"Nextsource Biotechnology, LLC",Released
7,Jun 23 2022 11:38AM,240294,10,SOTAH0000713,"Nextsource Biotechnology, LLC",Released
8,Jun 23 2022 11:38AM,240294,10,SOTAH0000715,"Nextsource Biotechnology, LLC",Released
9,Jun 23 2022 11:14AM,240293,10,0085798033,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,240292,Released,1
36,240293,Released,10
37,240294,Released,7
38,240295,Released,1
39,240296,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240292,NaN,NaN,1.0
240293,NaN,NaN,10.0
240294,NaN,NaN,7.0
240295,NaN,NaN,1.0
240296,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240135,0.0,0.0,1.0
240194,0.0,0.0,1.0
240201,0.0,0.0,1.0
240202,5.0,1.0,1.0
240211,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240135,0,0,1
240194,0,0,1
240201,0,0,1
240202,5,1,1
240211,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240135,0,0,1
1,240194,0,0,1
2,240201,0,0,1
3,240202,5,1,1
4,240211,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240135,,,1
1,240194,,,1
2,240201,,,1
3,240202,5,1,1
4,240211,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 23 2022 11:46AM,240296,10,Bio-PRF
1,Jun 23 2022 11:40AM,240295,21,"NBTY Global, Inc."
2,Jun 23 2022 11:38AM,240294,10,"Nextsource Biotechnology, LLC"
9,Jun 23 2022 11:14AM,240293,10,ISDIN Corporation
19,Jun 23 2022 11:11AM,240291,10,ISDIN Corporation
48,Jun 23 2022 11:11AM,240292,12,Hush Hush
49,Jun 23 2022 11:10AM,240289,10,ISDIN Corporation
74,Jun 23 2022 11:09AM,240288,10,ISDIN Corporation
77,Jun 23 2022 11:01AM,240286,10,ISDIN Corporation
88,Jun 23 2022 10:56AM,240285,19,"Granules Pharmaceuticals, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 23 2022 11:46AM,240296,10,Bio-PRF,,,1
1,Jun 23 2022 11:40AM,240295,21,"NBTY Global, Inc.",,,1
2,Jun 23 2022 11:38AM,240294,10,"Nextsource Biotechnology, LLC",,,7
3,Jun 23 2022 11:14AM,240293,10,ISDIN Corporation,,,10
4,Jun 23 2022 11:11AM,240291,10,ISDIN Corporation,,,29
5,Jun 23 2022 11:11AM,240292,12,Hush Hush,,,1
6,Jun 23 2022 11:10AM,240289,10,ISDIN Corporation,,,25
7,Jun 23 2022 11:09AM,240288,10,ISDIN Corporation,,,3
8,Jun 23 2022 11:01AM,240286,10,ISDIN Corporation,,6,5
9,Jun 23 2022 10:56AM,240285,19,"Granules Pharmaceuticals, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 11:46AM,240296,10,Bio-PRF,1,,
1,Jun 23 2022 11:40AM,240295,21,"NBTY Global, Inc.",1,,
2,Jun 23 2022 11:38AM,240294,10,"Nextsource Biotechnology, LLC",7,,
3,Jun 23 2022 11:14AM,240293,10,ISDIN Corporation,10,,
4,Jun 23 2022 11:11AM,240291,10,ISDIN Corporation,29,,
5,Jun 23 2022 11:11AM,240292,12,Hush Hush,1,,
6,Jun 23 2022 11:10AM,240289,10,ISDIN Corporation,25,,
7,Jun 23 2022 11:09AM,240288,10,ISDIN Corporation,3,,
8,Jun 23 2022 11:01AM,240286,10,ISDIN Corporation,5,6,
9,Jun 23 2022 10:56AM,240285,19,"Granules Pharmaceuticals, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 11:46AM,240296,10,Bio-PRF,1,,
1,Jun 23 2022 11:40AM,240295,21,"NBTY Global, Inc.",1,,
2,Jun 23 2022 11:38AM,240294,10,"Nextsource Biotechnology, LLC",7,,
3,Jun 23 2022 11:14AM,240293,10,ISDIN Corporation,10,,
4,Jun 23 2022 11:11AM,240291,10,ISDIN Corporation,29,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 11:46AM,240296,10,Bio-PRF,1.0,NaN,NaN
1,Jun 23 2022 11:40AM,240295,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jun 23 2022 11:38AM,240294,10,"Nextsource Biotechnology, LLC",7.0,NaN,NaN
3,Jun 23 2022 11:14AM,240293,10,ISDIN Corporation,10.0,NaN,NaN
4,Jun 23 2022 11:11AM,240291,10,ISDIN Corporation,29.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 11:46AM,240296,10,Bio-PRF,1.0,0.0,0.0
1,Jun 23 2022 11:40AM,240295,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jun 23 2022 11:38AM,240294,10,"Nextsource Biotechnology, LLC",7.0,0.0,0.0
3,Jun 23 2022 11:14AM,240293,10,ISDIN Corporation,10.0,0.0,0.0
4,Jun 23 2022 11:11AM,240291,10,ISDIN Corporation,29.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3363703,90.0,22.0,0.0
12,480575,1.0,1.0,0.0
15,961105,18.0,4.0,0.0
16,960940,4.0,0.0,0.0
19,720792,4.0,0.0,0.0
20,1201212,17.0,5.0,5.0
21,480506,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3363703,90.0,22.0,0.0
1,12,480575,1.0,1.0,0.0
2,15,961105,18.0,4.0,0.0
3,16,960940,4.0,0.0,0.0
4,19,720792,4.0,0.0,0.0
5,20,1201212,17.0,5.0,5.0
6,21,480506,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,90.0,22.0,0.0
1,12,1.0,1.0,0.0
2,15,18.0,4.0,0.0
3,16,4.0,0.0,0.0
4,19,4.0,0.0,0.0
5,20,17.0,5.0,5.0
6,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,90.0
1,12,Released,1.0
2,15,Released,18.0
3,16,Released,4.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,5.0,0.0
Executing,22.0,1.0,4.0,0.0,0.0,5.0,0.0
Released,90.0,1.0,18.0,4.0,4.0,17.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,5.0,0.0
1,Executing,22.0,1.0,4.0,0.0,0.0,5.0,0.0
2,Released,90.0,1.0,18.0,4.0,4.0,17.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,5.0,0.0
1,Executing,22.0,1.0,4.0,0.0,0.0,5.0,0.0
2,Released,90.0,1.0,18.0,4.0,4.0,17.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()